Pedido: Obtener NER y POS sobre el mismo dataset sobre todas las conversaciones etiquetadas manualmente.

Los datasets son:

- ecu_50_llamadas_sample_annotated.xlsx
    Primera muestra de 50 llamadas etiquetadas por el pasante mejia.

- ecu_50_llamadas_sample_annotated_v2.xlsx
    Segunda muestra de 50 llamadas etiquetada en conjunto por mi y por el pasante mejia.

- ecu_150_llamadas_sample_annotated_v3.xlsx
    Tercera muestra pero esta vez de 150 llamadas etiquetadas por Andres Sebastian Guerrero Zeas

En total se tienen 250 llamadas etiquetadas con acto de diálogo.

In [1]:
!which python

/root/bert_ft_marcos/ner_pos_tf_idf/.venv/bin/python


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

Spacy tiene opcion para NER pero no con un modelo
Transformer sino con "es_core_news_lg".
El modelo transformer "es_dep_news_trf" puede hacer POS

In [11]:
# !python -m spacy download es_core_news_lg

In [3]:
import spacy
spacy.require_gpu()

True

In [4]:
# Ver los componentes del modelo transformer
nlp = spacy.load("es_dep_news_trf")
print(nlp.pipe_names)

/root/bert_ft_marcos/ner_pos_tf_idf/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.

['transformer', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer']


In [3]:
# obtener NER
nlp = spacy.load("es_core_news_lg")
doc = nlp("La compañía UDA está lanzando un nuevo libro llamado 'Aprendiendo' del autor Santiago")

for ent in doc.ents:
    print(ent.text, ent.label_)

UDA ORG
Santiago PER


In [4]:
# Obtener POS
nlp = spacy.load("es_dep_news_trf")

doc = nlp("La empresa UDA publicó un libro nuevo.")

for token in doc:
    print(token.text, token.pos_, token.dep_)


La DET det
empresa NOUN nsubj
UDA PROPN appos
publicó VERB ROOT
un DET det
libro NOUN obj
nuevo ADJ amod
. PUNCT punct


In [3]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_excel("../datasets/ecu_250_sample_annotated_union_v1v2v3.xlsx")

In [4]:
df.head(5)

,TRA_ID,INCIDENTGRADENAME,ORDER,ATA_TEXTO_TAGGED,ACTOR,DIALOG_ACT
0,4365,CLAVE ROJA,1,emergencia,Operador,Pregunta
1,4365,CLAVE ROJA,2,buenos dias hagame un favor me puede mandar un...,Alertante,Pregunta
2,4365,CLAVE ROJA,3,que sucede señore que le sucede,Operador,Pregunta
3,4365,CLAVE ROJA,4,esta un carro parqueado y un par de jovenes co...,Alertante,Descripción
4,4365,CLAVE ROJA,5,le agredieron,Operador,Pregunta


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

spacy.require_gpu()

nlp_ner = spacy.load("es_core_news_lg")
nlp_pos = spacy.load("es_dep_news_trf")

def extract_ner(text):
    if not isinstance(text, str) or text.strip() == "":
        return []
    doc = nlp_ner(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

def extract_pos(text):
    if not isinstance(text, str) or text.strip() == "":
        return []
    doc = nlp_pos(text)
    return [(token.text, token.pos_) for token in doc]

df["NER"] = df["ATA_TEXTO_TAGGED"].progress_apply(extract_ner)
df["POS"] = df["ATA_TEXTO_TAGGED"].progress_apply(extract_pos)

# stop words desde spacy

#stopwords_es = nlp_ner.Defaults.stop_words

# TF-IDF
# Devuelve una matriz (sparse matrix) de forma:
# (n_documentos, n_features)
# n_documentos = filas dataset
# n_features palabras más frecuentes, definidas por max_features
# vectorizer = TfidfVectorizer(
#     lowercase=True,
#     stop_words=list(stopwords_es),
#     ngram_range=(1,2),
#     max_features=3000, # Número máximo de columnas (palabras) a conservar.
#     # min_df=5      # ignora palabras que aparecen en menos de 5 documentos
#     # max_df=0.8    # ignora palabras que aparecen en >80% de los documentos
# )

#tfidf_matrix = vectorizer.fit_transform(df["ATA_TEXTO_TAGGED"].fillna(""))


/root/bert_ft_marcos/ner_pos_tf_idf/.venv/lib/python3.10/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.

In [6]:
df

,TRA_ID,INCIDENTGRADENAME,ORDER,ATA_TEXTO_TAGGED,ACTOR,DIALOG_ACT,NER,POS
0,4365,CLAVE ROJA,1,emergencia,Operador,Pregunta,[],"[(emergencia, NOUN)]"
1,4365,CLAVE ROJA,2,buenos dias hagame un favor me puede mandar un...,Alertante,Pregunta,"[(buenos dias hagame, MISC)]","[(buenos, ADJ), (dias, NOUN), (hagame, VERB), ..."
2,4365,CLAVE ROJA,3,que sucede señore que le sucede,Operador,Pregunta,"[(señore, PER)]","[(que, SCONJ), (sucede, VERB), (señore, VERB),..."
3,4365,CLAVE ROJA,4,esta un carro parqueado y un par de jovenes co...,Alertante,Descripción,[],"[(esta, VERB), (un, DET), (carro, NOUN), (parq..."
4,4365,CLAVE ROJA,5,le agredieron,Operador,Pregunta,[],"[(le, PRON), (agredieron, VERB)]"
...,...,...,...,...,...,...,...,...
5047,3984,CLAVE NARANJA,14,estan abordo de un vehiculo estan tres persona...,Alertante,Descripción,[],"[(estan, VERB), (abordo, ADV), (de, ADP), (un,..."
5048,3984,CLAVE NARANJA,15,cual es su nombre por favor,Operador,Pregunta,[],"[(cual, PRON), (es, AUX), (su, DET), (nombre, ..."
5049,3984,CLAVE NARANJA,16,miriam mejia,Alertante,Descripción,"[(miriam mejia, PER)]","[(miriam, PROPN), (mejia, PROPN)]"
5050,3984,CLAVE NARANJA,17,coordinamos su emergencia estamos para servile,Operador,Rutina,"[(servile, LOC)]","[(coordinamos, VERB), (su, DET), (emergencia, ..."


In [7]:
a = df.iloc[0]["POS"]

In [8]:
for e in a:
    print(e)
    print(type(e))

('emergencia', 'NOUN')
<class 'tuple'>


In [ ]:
# df.to_csv("250_conversaciones_ner_pos.csv", index=False)
# df.to_excel("250_conversaciones_ner_pos.xlsx", index=False)